In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
import joblib
warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer, PowerTransformer
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, SGDRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_validate, StratifiedKFold, KFold, GridSearchCV, LearningCurveDisplay
from sklearn.metrics import PredictionErrorDisplay, root_mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree

from utils import plot_outlier_analysis, residual_plots

pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
dataset = pd.read_csv('train.csv')

y = dataset[['SalePrice']]
log1p_y = np.log1p(y)
X = dataset.drop(['SalePrice', 'Id'], axis = 1)

testset = pd.read_csv('test.csv')


In [3]:
chg_to_category = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
                   'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 
                   'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                   'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                   'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 
                   'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                   'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 
                   'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
                   'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
                   'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 
                'SaleType', 'SaleCondition']

chg_to_numer = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 
                'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
                'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 
                'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
                'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 
                'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
                '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'YrSold',
                'MoSold'
                ]

numerical_selector = make_column_selector(dtype_include=['int64', 'float64'])
cat_selector = make_column_selector(dtype_include=['object'])

X[chg_to_numer] = X[chg_to_numer].astype('float64')
X[chg_to_category] = X[chg_to_category].astype('object')

testset[chg_to_numer] = testset[chg_to_numer].astype('float64')
testset[chg_to_category] = testset[chg_to_category].astype('object')

continous_num = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                 '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                 'EnclosedPorch', '3SsnPorch',	'ScreenPorch',	'PoolArea',	'MiscVal']

time_based_num = ['MoSold', 'YrSold', 'GarageYrBlt', 'YearRemodAdd', 'YearBuilt']

discrete_num = list(set(chg_to_numer) - set(continous_num) - set(time_based_num))


In [19]:
#FEATURE ENGINEERING

# pd.set_option('display.max_columns', 500)
# X[time_based_num]
# X.loc[:, 'age_at_remodel'] = X['YearRemodAdd'] - X['YearBuilt']
# X.loc[:, 'age_at_sell'] = X['YrSold'] - X['YearBuilt']

X_fortree = pd.get_dummies(X)

dtr = DecisionTreeRegressor().fit(X_fortree, y)
dtr
# dir(dtr.tree_)
# dtr.tree_.impurity
# plot_tree(dtr, max_depth = 10, feature_names = dtr.feature_names_in_)
# plt.savefig('tree.pdf')

# cv_res = cross_validate(DecisionTreeRegressor(random_state=0),
#                pd.get_dummies(X), y,
#                scoring='neg_root_mean_squared_error',
#                return_train_score=True)

# pd.DataFrame(cv_res)

X[X['Exterior1st'] == 'MetalSd'].describe()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,189.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,...,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000
mean,62.810,8611.623,5.532,5.845,1956.241,1976.682,82.991,387.523,33.736,504.305,...,412.145,62.032,33.750,35.932,0.000,12.886,0.000,113.000,6.450,2007.755
std,19.090,4213.934,1.229,1.128,28.571,22.237,169.648,373.764,128.409,342.343,...,214.892,104.998,56.553,73.311,0.000,49.414,0.000,1064.408,2.635,1.266
min,24.000,1300.000,3.000,2.000,1872.000,1950.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,2006.000
25%,50.000,6240.000,5.000,5.000,1938.000,1952.250,0.000,0.000,0.000,261.750,...,280.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,2007.000
50%,60.000,8402.500,5.000,6.000,1956.000,1973.500,0.000,367.000,0.000,468.000,...,404.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,6.000,2008.000
75%,73.000,10328.750,6.000,7.000,1971.250,2000.000,116.250,604.750,0.000,747.250,...,528.000,120.000,50.000,0.000,0.000,0.000,0.000,0.000,8.000,2009.000
max,138.000,35133.000,9.000,9.000,2009.000,2009.000,1129.000,1696.000,1029.000,1869.000,...,1356.000,495.000,285.000,330.000,0.000,288.000,0.000,15500.000,12.000,2010.000


In [10]:
# PREPROCESSING TIMELINE

def mod_ordinal_transform(X, imputer):

    missing_imputer = imputer

    testset = pd.read_csv("test.csv")
    trainset = pd.read_csv("train.csv")

    full_data = pd.concat([trainset, testset])

    #Fitting imputer on test+train set for consistent ordinal encoding
    missing_imputer.fit(full_data[chg_to_category])
    imputed_values_fulldata = pd.DataFrame(missing_imputer.transform(full_data[chg_to_category]))
    imputed_values_fulldata.columns = missing_imputer.get_feature_names_out()

    enc = OrdinalEncoder().fit(imputed_values_fulldata)


    X_imputed = pd.DataFrame(missing_imputer.transform(X[chg_to_category]))
    X_imputed.columns = missing_imputer.get_feature_names_out()

    X_transf = pd.DataFrame(enc.transform(X_imputed))

    X_transf.columns = enc.get_feature_names_out()
    X_transf.index = X.index
    X_transf = X_transf[chg_to_category]

    return X_transf


preprocessors = ColumnTransformer(
    [("scaler", StandardScaler(), numerical_selector),
     ("enc", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_selector)]
    ).set_output(transform='pandas')

preprocessors_2 = ColumnTransformer(
    [("scaler", StandardScaler(), numerical_selector),
     ("enc", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_selector)]
    ).set_output(transform='pandas')

preprocessors_3 = ColumnTransformer(
    [("scaler", StandardScaler(), numerical_selector),
     ("enc", OneHotEncoder(handle_unknown='infrequent_if_exist'), cat_selector)]
    )

preprocessors_4 = ColumnTransformer(
    [("scaler", PowerTransformer(method='yeo-johnson', standardize=True), numerical_selector),
     ("enc", OneHotEncoder(handle_unknown='infrequent_if_exist'), cat_selector)]
    )


preprocessors_6 = ColumnTransformer(
    transformers= [
        ("scaler", StandardScaler(), numerical_selector),
        ("enc", FunctionTransformer(mod_ordinal_transform, 
                                    validate=False, 
                                    kw_args={'imputer': SimpleImputer(strategy='constant', fill_value='not available')}), 
                                    chg_to_category)
    ],
    verbose_feature_names_out=False
).set_output(transform='pandas')

preprocessors_7 = ColumnTransformer(
    transformers=[
        ("cont_num", PowerTransformer(method='yeo-johnson', standardize=True), continous_num),
        ("time_num", StandardScaler(), time_based_num),
        ("discrete_num", OneHotEncoder(handle_unknown='infrequent_if_exist'), discrete_num),
        ("enc", OneHotEncoder(handle_unknown='infrequent_if_exist'), chg_to_category)
    ],
    verbose_feature_names_out = False
)

# pd.DataFrame(preprocessors.fit_transform(X))

missing_imputer = ColumnTransformer(
    [("num_mean", SimpleImputer(strategy='mean'), numerical_selector),
     ("cat_ordinal", SimpleImputer(strategy='most_frequent'), cat_selector)]
    ).set_output(transform='pandas')


missing_imputer_2 = ColumnTransformer(
    [("num_mean", SimpleImputer(strategy = 'constant', fill_value=0), chg_to_numer),
     ("cat_ordinal", SimpleImputer(strategy='constant', fill_value='NA'), chg_to_category)]
    ).set_output(transform='pandas')

missing_imputer_3 = ColumnTransformer(
    [("num_mean", KNNImputer(n_neighbors=5), chg_to_numer),
     ("cat_ordinal", SimpleImputer(strategy='constant', fill_value='not available'), chg_to_category)],
        verbose_feature_names_out = False
    ).set_output(transform='pandas')

missing_imputer_4 = ColumnTransformer(
    [("num_mean", KNNImputer(n_neighbors=5), chg_to_numer),
     ("cat_ordinal", SimpleImputer(strategy='most_frequent'), chg_to_category)],
     verbose_feature_names_out=False
    ).set_output(transform='pandas')

missing_imputer_5 = ColumnTransformer(
    [("num_mean", SimpleImputer(strategy='mean'), chg_to_numer),
     ("cat_ordinal", SimpleImputer(strategy='most_frequent'), chg_to_category)]
    ).set_output(transform='pandas')


# pd.DataFrame(preprocessors_2.fit_transform(X))



In [ ]:
# GRIDSEARCH FOR PREPROCESSING AND ESTIMATOR PIPELINE 

linearReg_pipe = Pipeline([
    ('missing_imputer', missing_imputer),
    ('preprocessor', preprocessors),
    ('linearreg', LinearRegression())
])

# Define the parameter grid
# param_grid = [{
#     'missing_imputer': [missing_imputer, missing_imputer_2, missing_imputer_3, missing_imputer_4, missing_imputer_5],
#     'preprocessor': [preprocessors, preprocessors_2, preprocessors_3, preprocessors_4, preprocessors_6],
#     'linearreg': [LinearRegression(), RidgeCV(), Ridge()]
# }]

imputers = [missing_imputer_2, missing_imputer_3, missing_imputer_4, missing_imputer_5]
preprocesrs = [preprocessors_2, preprocessors_3, preprocessors_4, preprocessors_6, preprocessors_7]

param_grid = [
    {
        'missing_imputer': imputers,
        'preprocessor': preprocesrs,
        'linearreg': [LinearRegression()],
        'linearreg__fit_intercept': [True, False]
    },
    {
        'missing_imputer': imputers,
        'preprocessor': preprocesrs,
        'linearreg': [RidgeCV()],
        'linearreg__alphas': [(0.1, 1.0, 10.0), (0.01, 0.1, 1.0, 10.0), (0.001, 0.01, 0.1, 10, 100.0)],
        'linearreg__fit_intercept': [True, False],
        'linearreg__gcv_mode': ['auto', 'svd', 'eigen']
    },
    {
        'missing_imputer': imputers,
        'preprocessor': preprocesrs,
        'linearreg': [Ridge()],
        'linearreg__alpha': [0.1, 1.0, 10.0, 100.0],
        'linearreg__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    },
    {
        'missing_imputer': imputers,
        'preprocessor': preprocesrs,
        'linearreg': [SGDRegressor()],
        'linearreg__alpha': np.linspace(0.00001, 10, num=5),
        'linearreg__penalty': ['elasticnet'],
        'linearreg__l1_ratio': np.linspace(0.01, 1, num=5)
    }
]

# Set up and fit the GridSearchCV
grid_search = GridSearchCV(linearReg_pipe, 
                           param_grid, 
                           cv=5, scoring='neg_root_mean_squared_error', 
                           return_train_score=True,
                           verbose=1,
                           n_jobs=-1)
grid_search.fit(X, log1p_y)

# Get the best parameters and scores
print("Best parameters found:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)
pd.DataFrame(grid_search.cv_results_).sort_values(by=['rank_test_score'])


In [20]:
# manual cross fold

# kf = KFold(n_splits=5, shuffle=False)

# for i, (train_index, test_index) in enumerate(kf.split(dataset)):
#     dataset.loc[test_index, 'kfold'] = i

# for i in dataset['kfold'].unique():
#     validation_set = dataset[dataset['kfold'] == i]
#     train_set = dataset[dataset['kfold'] != i]
#     y_train = train_set[['SalePrice']]
#     y_validation = validation_set[['SalePrice']]

#     validation_set = validation_set.drop(columns=['Id', 'SalePrice', 'kfold'])
#     train_set = train_set.drop(columns=['Id', 'SalePrice', 'kfold'])

#     model_pipe.fit(train_set, y_train)

#     # if i == 3:
#     #     display(model_pipe[:-1].fit_transform(validation_set)['Utilities'].value_counts())

#     #     display(pd.DataFrame(model_pipe[2].coef_))
#     #     display(model_pipe[:-1].fit_transform(validation_set))

#     display(f"""fold: {i}, 
#             train_score: {model_pipe.score(train_set, y_train)}, 
#             test_score:{model_pipe.score(validation_set, y_validation)},
#             rmse = {root_mean_squared_error(y_validation, model_pipe.predict(validation_set))}""")


In [ ]:
# Residual Plots for comparison

# display(reg_model.score(X, y))
# residual_plots(model = reg_model, data=X, y=y)

# display(reg_model_2.score(X, y))
# residual_plots(model = reg_model_2, data=X, y=y)

# display(reg_model_3.score(X, y))
# residual_plots(model = reg_model_3, data=X, y=y)

# # Best So FAR: reg_model_4

# display(reg_model_4.score(X, y))
# residual_plots(model = reg_model_4, data=X, y=y)

# display(reg_model_5.score(X, y))
# residual_plots(model = reg_model_5, data=X, y=y)

best_gridsearchCV_pipe = Pipeline([
    ('missing_imputer', grid_search.best_params_['missing_imputer']),
    ('preprocessor', grid_search.best_params_['preprocessor']),
    ('linearreg', grid_search.best_params_['linearreg'])
])
best_gridsearchCV_pipe.fit(X, log1p_y)
display(best_gridsearchCV_pipe.score(X, log1p_y))
residual_plots(model = best_gridsearchCV_pipe, data=X, y=log1p_y)
# grid_search.best_params_
# pd.DataFrame(best_gridsearchCV_pipe[1].fit_transform(X).toarray())
# best_gridsearchCV_pipe[0].fit_transform(X)
# X


In [ ]:

cv_res = cross_validate(best_gridsearchCV_pipe,
                        X, log1p_y,
                        scoring=('neg_root_mean_squared_error', 'r2'),
                        cv = 5, return_train_score=True)
cv_res = pd.DataFrame(cv_res)
display(f"mean_test : {cv_res['test_neg_root_mean_squared_error'].mean()}, mean_train : {cv_res['train_neg_root_mean_squared_error'].mean()}, {cv_res['test_r2'].mean()}; {cv_res['train_r2'].mean()}")

In [21]:
# subm = testset[['Id']].join(pd.DataFrame(best_gridsearchCV_pipe.predict(testset), columns=['SalePrice']))
# subm[['SalePrice']] = np.expm1(subm[['SalePrice']])
# subm.to_csv('submissions/grid_search_3_imp2_p4_ridge.csv')

In [ ]:
#LEARNING CURVE OF MODEL



# Create train sizes (10% to 100%)
train_sizes = np.linspace(0.1, 1.0, 10)


LearningCurveDisplay.from_estimator(
    best_gridsearchCV_pipe,
    X, y,
    train_sizes= train_sizes,
    scoring = 'r2',
    std_display_style = 'errorbar'
)

In [ ]:
# MLE Experiment

# import numpy as np
# import pandas as pd
# from scipy.optimize import minimize
# from scipy.stats import t

# # Generate synthetic data
# np.random.seed(42)
# X = np.random.normal(size=(100, 1))
# beta = np.array([2.5])
# epsilon = np.random.standard_t(df=3, size=100)  # t-distribution errors
# y = X.dot(beta) + epsilon

# # Define the log-likelihood function for t-distribution
# def log_likelihood(params):
#     beta = params[:-1]
#     df = params[-1]
#     residuals = y - X.dot(beta)
    
#     likelihood = np.prod(t.pdf(residuals, df=df)) 
#     print(f"Likelihood: {likelihood}")

#     log_likelihood = np.sum(t.logpdf(residuals, df=df))
#     return -log_likelihood

# # Initial guess for parameters
# initial_params = np.append(np.ones(X.shape[1]), 3.0)

# # Optimize the log-likelihood function
# result = minimize(log_likelihood, initial_params, method='BFGS')  # You can try different methods like 'L-BFGS-B', 'Nelder-Mead', etc.

# # Extract estimated parameters
# beta_est = result.x[:-1]
# df_est = result.x[-1]

# print(f"Estimated beta: {beta_est}")
# print(f"Estimated degrees of freedom: {df_est}")
# log_likelihood(initial_params)